In [1]:
import numpy as np
import pandas as pd

admissions = pd.read_csv('02-02-08-binary.csv')

# Make dummy variables for rank
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)

# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std
    
# Split off random 10% of the data for testing
np.random.seed(21)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.iloc[sample,:], data.drop(sample)

# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

In [2]:
np.random.seed(21)

def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))


# Hyperparameters
n_hidden = 2  # number of hidden units
epochs = 900
learnrate = 0.005

n_records, n_features = features.shape
last_loss = None
# Initialize weights
weights_input_hidden_init = np.random.normal(scale=1 / n_features ** .5,
                                        size=(n_features, n_hidden))
weights_hidden_output_init = np.random.normal(scale=1 / n_features ** .5,
                                         size=n_hidden)



weights_input_hidden = weights_input_hidden_init.copy()
weights_hidden_output = weights_hidden_output_init.copy()

for e in range(epochs):
    del_w_input_hidden = np.zeros(weights_input_hidden.shape)
    del_w_hidden_output = np.zeros(weights_hidden_output.shape)
    for x, y in zip(features.values, targets):
        ## Forward pass ##
        # TODO: Calculate the output
        hidden_input = x
        hidden_output = sigmoid(np.dot(x, weights_input_hidden))
        output = sigmoid(np.dot(hidden_output, weights_hidden_output))

        ## Backward pass ##
        # TODO: Calculate the network's prediction error
        error = y-output

        # TODO: Calculate error term for the output unit
        output_error_term = error * output * (1-output)

        ## propagate errors to hidden layer

        # TODO: Calculate the hidden layer's contribution to the error
        hidden_error = np.dot(output_error_term, weights_hidden_output)
        
        # TODO: Calculate the error term for the hidden layer
        hidden_error_term = hidden_error*hidden_output*(1-hidden_output)
        
        # TODO: Update the change in weights
        del_w_hidden_output += output_error_term * hidden_output
        del_w_input_hidden += hidden_error_term * x[:,None]

    # TODO: Update weights  (don't forget to division by n_records or number of samples)
    weights_input_hidden += learnrate/n_records*del_w_input_hidden
    weights_hidden_output += learnrate/n_records*del_w_hidden_output
    

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        hidden_output = sigmoid(np.dot(x, weights_input_hidden))
        out = sigmoid(np.dot(hidden_output,
                             weights_hidden_output))
        loss = np.mean((out - targets) ** 2)

        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss

# Calculate accuracy on test data
hidden = sigmoid(np.dot(features_test, weights_input_hidden))
out = sigmoid(np.dot(hidden, weights_hidden_output))
predictions = out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

Train loss:  0.28521964587875515
Train loss:  0.2838619826548728
Train loss:  0.28253760593234994
Train loss:  0.2812460138211576
Train loss:  0.27998668772050733
Train loss:  0.2787590944032222
Train loss:  0.27756268800400163
Train loss:  0.27639691190999127
Train loss:  0.2752612005528652
Train loss:  0.274154981102325
Prediction accuracy: 0.667


In [3]:
# same implementation mais en vectoriel
np.random.seed(21)

# Hyperparameters
n_hidden = 2  # number of hidden units
epochs = 900
learnrate = 0.005

n_records, n_features = features.shape
last_loss = None
# Initialize weights
weights_input_hidden = weights_input_hidden_init.copy()
weights_hidden_output = weights_hidden_output_init.copy()

for e in range(epochs):
    del_w_input_hidden = np.zeros(weights_input_hidden.shape)
    del_w_hidden_output = np.zeros(weights_hidden_output.shape)
    
    ## Forward pass ##
    # TODO: Calculate the output
    hidden_output = sigmoid(features @ weights_input_hidden)

    output = sigmoid(hidden_output @ weights_hidden_output)
    error = targets-output
    output_error_term = error * output * (1-output)

    hidden_error = output_error_term[:,None] @ weights_hidden_output[:,None].T
    hidden_error_term = hidden_error*hidden_output*(1-hidden_output)
    del_w_hidden_output = hidden_output.mul(output_error_term, axis=0).mean()
    del_w_input_hidden[:,0] = features.mul(hidden_error_term[0], axis=0).mean()
    del_w_input_hidden[:,1] = features.mul(hidden_error_term[1], axis=0).mean()

    # TODO: Update weights  (don't forget to division by n_records or number of samples)
    weights_input_hidden = weights_input_hidden + learnrate*del_w_input_hidden
    weights_hidden_output =  + learnrate*del_w_hidden_output

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        loss = np.mean((output - targets) ** 2)

        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss

# Calculate accuracy on test data
hidden = sigmoid(np.dot(features_test, weights_input_hidden))
out = sigmoid(np.dot(hidden, weights_hidden_output))
predictions = out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

Train loss:  0.26152732708624066
Train loss:  0.24999187612205562
Train loss:  0.24999187611844653
Train loss:  0.24999187611483756
Train loss:  0.2499918761112286
Train loss:  0.24999187610761958
Train loss:  0.24999187610401044
Train loss:  0.24999187610040155
Train loss:  0.2499918760967924
Train loss:  0.24999187609318346
Prediction accuracy: 0.778
